# RAS Commander: Unsteady Flow Operations

This notebook demonstrates operations on unsteady flow files using the RAS Commander library. Unsteady flow files in HEC-RAS (`.u*` files) define the time-varying boundary conditions used in dynamic simulations.

## Operations Covered

1. **Project Initialization**: Initialize a HEC-RAS project by specifying the project path and version
2. **Boundary Extraction**: Extract boundary conditions and tables from unsteady flow files
3. **Boundary Analysis**: Inspect and understand boundary condition structures
4. **Flow Title Updates**: Modify the title of unsteady flow files
5. **Restart Settings**: Configure restart file settings for continuing simulations
6. **Table Modification**: Extract, modify, and update flow tables

Let's begin by importing the necessary libraries and setting up our environment.

In [1]:
# 1. Install ras-commander from pip (uncomment to install if needed)
#!pip install ras-commander
# This installs ras-commander and all dependencies

In [2]:
# 2. Import all required modules

# Import all ras-commander modules
from ras_commander import *

# Import the required libraries for this notebook
import numpy as np
import pandas as pd
from IPython import display
import os
from pathlib import Path
import matplotlib.pyplot as plt

## Understanding Unsteady Flow Files in HEC-RAS

Unsteady flow files (`.u*` files) in HEC-RAS define the time-varying boundary conditions that drive dynamic simulations. These include:

- **Flow Hydrographs**: Time-series of flow values at model boundaries
- **Stage Hydrographs**: Time-series of water surface elevations
- **Lateral Inflows**: Distributed inflows along a reach
- **Gate Operations**: Time-series of gate settings
- **Meteorological Data**: Rainfall, evaporation, and other meteorological inputs

The `RasUnsteady` class in RAS Commander provides methods for working with these files, including extracting boundaries, reading tables, and modifying parameters.

Let's set up our working directory and define paths to example projects:

## Downloading and Extracting Example HEC-RAS Projects

We'll use the `RasExamples` class to download and extract an example HEC-RAS project with unsteady flow files. For this notebook, we'll use the "Balde Eagle Creek" project.

In [ ]:
# Extract the Bald Eagle Creek example project
# The extract_project method downloads the project from GitHub if not already present,
# and extracts it to the example_projects folder
bald_eagle_path = RasExamples.extract_project("Balde Eagle Creek")
print(f"Extracted project to: {bald_eagle_path}")  


# Verify the path exists
print(f"Bald Eagle Creek project exists: {bald_eagle_path.exists()}")

## Step 1: Project Initialization

The first step is to initialize the HEC-RAS project. This is done using the `init_ras_project()` function, which takes the following parameters:

- `ras_project_folder`: Path to the HEC-RAS project folder (required)
- `ras_version`: HEC-RAS version (e.g., "6.6") or path to Ras.exe (required first time)

This function initializes the global `ras` object that we'll use for the rest of the notebook.

In [ ]:
# Initialize the HEC-RAS project
# This function returns a RAS object, but also updates the global 'ras' object
# Parameters:
#   - ras_project_folder: Path to the HEC-RAS project folder
#   - ras_version: HEC-RAS version or path to Ras.exe

init_ras_project(bald_eagle_path, "6.6")
print(f"Initialized HEC-RAS project: {ras.project_name}")

# Display the unsteady flow files in the project
print("\nHEC-RAS Project Unsteady Flow Data (unsteady_df):")
display.display(ras.unsteady_df)

## Understanding the RasUnsteady Class

The `RasUnsteady` class provides functionality for working with HEC-RAS unsteady flow files (`.u*` files). Key operations include:

1. **Extracting Boundary Conditions**: Read and parse boundary conditions from unsteady flow files
2. **Modifying Flow Titles**: Update descriptive titles for unsteady flow scenarios
3. **Managing Restart Settings**: Configure restart file options for continuing simulations
4. **Working with Tables**: Extract, modify, and update flow tables

Most methods in this class are static and work with the global `ras` object by default, though you can also pass in a custom RAS object.

## Step 2: Extract Boundary Conditions and Tables

The `extract_boundary_and_tables()` method from the `RasUnsteady` class allows us to extract boundary conditions and their associated tables from an unsteady flow file.

Parameters for `RasUnsteady.extract_boundary_and_tables()`:
- `unsteady_file` (str): Path to the unsteady flow file
- `ras_object` (optional): Custom RAS object to use instead of the global one

Returns:
- `pd.DataFrame`: DataFrame containing boundary conditions and their associated tables

Let's see how this works with our example project:

In [ ]:
# Get the path to unsteady flow file "02"
unsteady_file = RasPlan.get_unsteady_path("02")
print(f"Unsteady flow file path: {unsteady_file}")

# Extract boundary conditions and tables
boundaries_df = RasUnsteady.extract_boundary_and_tables(unsteady_file)
print(f"Extracted {len(boundaries_df)} boundary conditions from the unsteady flow file.")


## Step 3: Print Boundaries and Tables

The `print_boundaries_and_tables()` method provides a formatted display of the boundary conditions and their associated tables. This method doesn't return anything; it just prints the information in a readable format.

Parameters for `RasUnsteady.print_boundaries_and_tables()`:
- `boundaries_df` (pd.DataFrame): DataFrame containing boundary conditions from `extract_boundary_and_tables()`

Let's use this method to get a better understanding of our boundary conditions:

In [ ]:
# Print the boundaries and tables in a formatted way
print("Detailed boundary conditions and tables:")
RasUnsteady.print_boundaries_and_tables(boundaries_df)

## Understanding Boundary Condition Types

The output above shows the different types of boundary conditions in our unsteady flow file. Let's understand what each type means:

1. **Flow Hydrograph**: A time series of flow values (typically in cfs or cms) entering the model at a specific location. These are used at upstream boundaries or internal points where flow enters the system.

2. **Stage Hydrograph**: A time series of water surface elevations (typically in ft or m) that define the downstream boundary condition.

3. **Gate Openings**: Time series of gate settings (typically height in ft or m) for hydraulic structures such as spillways, sluice gates, or other control structures.

4. **Lateral Inflow Hydrograph**: Flow entering the system along a reach, not at a specific point. This can represent tributary inflows, overland flow, or other distributed inputs.

5. **Normal Depth**: A boundary condition where the water surface slope is assumed to equal the bed slope. This is represented by a friction slope value.

Let's look at a specific boundary condition in more detail:

In [ ]:
# Let's examine the first boundary condition in more detail
if not boundaries_df.empty:
    first_boundary = boundaries_df.iloc[0]
    print(f"Detailed look at boundary condition {1}:")
    
    # Print boundary location components
    print(f"\nBoundary Location:")
    print(f"  River Name: {first_boundary.get('River Name', 'N/A')}")
    print(f"  Reach Name: {first_boundary.get('Reach Name', 'N/A')}")
    print(f"  River Station: {first_boundary.get('River Station', 'N/A')}")
    print(f"  Storage Area Name: {first_boundary.get('Storage Area Name', 'N/A')}")
    
    # Print boundary condition type and other properties
    print(f"\nBoundary Properties:")
    print(f"  Boundary Type: {first_boundary.get('bc_type', 'N/A')}")
    print(f"  DSS File: {first_boundary.get('DSS File', 'N/A')}")
    print(f"  Use DSS: {first_boundary.get('Use DSS', 'N/A')}")
    
    # Print table statistics if available
    if 'Tables' in first_boundary and isinstance(first_boundary['Tables'], dict):
        print(f"\nTable Information:")
        for table_name, table_df in first_boundary['Tables'].items():
            print(f"  {table_name}: {len(table_df)} values")
            if not table_df.empty:
                print(f"    Min Value: {table_df['Value'].min()}")
                print(f"    Max Value: {table_df['Value'].max()}")
                print(f"    First 5 Values: {table_df['Value'].head(5).tolist()}")
else:
    print("No boundary conditions found in the unsteady flow file.")

## Step 4: Update Flow Title

The flow title in an unsteady flow file provides a description of the simulation scenario. The `update_flow_title()` method allows us to modify this title.

Parameters for `RasUnsteady.update_flow_title()`:
- `unsteady_file` (str): Full path to the unsteady flow file
- `new_title` (str): New flow title (max 24 characters)
- `ras_object` (optional): Custom RAS object to use instead of the global one

Let's clone an unsteady flow file and update its title:

In [ ]:
# Clone unsteady flow "02" to create a new unsteady flow file
new_unsteady_number = RasPlan.clone_unsteady("02")
print(f"New unsteady flow created: {new_unsteady_number}")

# Get the path to the new unsteady flow file
new_unsteady_file = RasPlan.get_unsteady_path(new_unsteady_number)
print(f"New unsteady flow file path: {new_unsteady_file}")

# Get the current flow title
current_title = None
for _, row in ras.unsteady_df.iterrows():
    if row['unsteady_number'] == new_unsteady_number and 'Flow Title' in row:
        current_title = row['Flow Title']
        break
print(f"Current flow title: {current_title}")

# Update the flow title
new_title = "Modified Flow Scenario"
RasUnsteady.update_flow_title(new_unsteady_file, new_title)
print(f"Updated flow title to: {new_title}")

# Refresh unsteady flow information to see the change
ras.unsteady_df = ras.get_unsteady_entries()
display.display(ras.unsteady_df[ras.unsteady_df['unsteady_number'] == new_unsteady_number])

## Step 5: Update Restart Settings

Restart files in HEC-RAS allow you to continue a simulation from a specific point in time, which can save computational resources. The `update_restart_settings()` method allows you to configure restart options.

Parameters for `RasUnsteady.update_restart_settings()`:
- `unsteady_file` (str): Full path to the unsteady flow file
- `use_restart` (bool): Whether to use restart (True) or not (False)
- `restart_filename` (str, optional): Name of the restart file (required if use_restart is True)
- `ras_object` (optional): Custom RAS object to use instead of the global one

Let's update the restart settings for our new unsteady flow file:

In [ ]:
# Update restart settings
restart_filename = f"{ras.project_name}.rst"
RasUnsteady.update_restart_settings(new_unsteady_file, use_restart=True, restart_filename=restart_filename)
print(f"Updated restart settings to use restart file: {restart_filename}")

# Let's extract the boundaries again to see the updated settings
updated_boundaries_df = RasUnsteady.extract_boundary_and_tables(new_unsteady_file)

# Check if there's a "Use Restart" property in the updated file
use_restart_value = None
restart_filename_value = None
with open(new_unsteady_file, 'r') as file:
    for line in file:
        if line.startswith("Use Restart="):
            use_restart_value = line.strip().split('=')[1].strip()
        elif line.startswith("Restart Filename="):
            restart_filename_value = line.strip().split('=')[1].strip()

print(f"\nVerified restart settings:")
print(f"Use Restart: {use_restart_value}")
print(f"Restart Filename: {restart_filename_value}")

# DOES NOT WORK

## Step 6: Working with Flow Tables

Flow tables in unsteady flow files contain the time-series data for boundary conditions. Let's explore how to extract and work with these tables using some of the advanced methods from the `RasUnsteady` class.

In [ ]:
# Extract specific tables from the unsteady flow file
all_tables = RasUnsteady.extract_tables(new_unsteady_file)
print(f"Extracted {len(all_tables)} tables from the unsteady flow file.")

# Let's look at the available table names
print("\nAvailable tables:")
for table_name in all_tables.keys():
    print(f"  {table_name}")

# Select the first table for detailed analysis
if all_tables and len(all_tables) > 0:
    first_table_name = list(all_tables.keys())[0]
    first_table = all_tables[first_table_name]
    
    print(f"\nDetailed look at table '{first_table_name}':")
    print(f"  Number of values: {len(first_table)}")
    print(f"  Min value: {first_table['Value'].min()}")
    print(f"  Max value: {first_table['Value'].max()}")
    print(f"  Mean value: {first_table['Value'].mean():.2f}")
    print(f"  First 10 values: {first_table['Value'].head(10).tolist()}")
    
    # Create a visualization of the table values
    try:
        import matplotlib.pyplot as plt
        
        plt.figure(figsize=(10, 6))
        plt.plot(first_table['Value'].values)
        plt.title(f"{first_table_name} Values")
        plt.xlabel('Time Step')
        plt.ylabel('Value')
        plt.grid(True)
        plt.show()
    except Exception as e:
        print(f"Could not create visualization: {e}")
else:
    print("No tables found in the unsteady flow file.")

## Step 7: Modifying Flow Tables

Now let's demonstrate how to modify a flow table and write it back to the unsteady flow file. For this example, we'll scale all the values in a table by a factor.

In [ ]:
# First, identify tables in the unsteady flow file
tables = RasUnsteady.identify_tables(open(new_unsteady_file, 'r').readlines())
print(f"Identified {len(tables)} tables in the unsteady flow file.")

# Let's look at the first flow hydrograph table
flow_hydrograph_tables = [t for t in tables if t[0] == 'Flow Hydrograph=']
if flow_hydrograph_tables:
    table_name, start_line, end_line = flow_hydrograph_tables[0]
    print(f"\nSelected table: {table_name}")
    print(f"  Start line: {start_line}")
    print(f"  End line: {end_line}")
    
    # Parse the table
    lines = open(new_unsteady_file, 'r').readlines()
    table_df = RasUnsteady.parse_fixed_width_table(lines, start_line, end_line)
    print(f"\nOriginal table statistics:")
    print(f"  Number of values: {len(table_df)}")
    print(f"  Min value: {table_df['Value'].min()}")
    print(f"  Max value: {table_df['Value'].max()}")
    print(f"  First 5 values: {table_df['Value'].head(5).tolist()}")
    
    # Modify the table - let's scale all values by 75%
    scale_factor = 0.75
    table_df['Value'] = table_df['Value'] * scale_factor
    print(f"\nModified table statistics (scaled by {scale_factor}):")
    print(f"  Number of values: {len(table_df)}")
    print(f"  Min value: {table_df['Value'].min()}")
    print(f"  Max value: {table_df['Value'].max()}")
    print(f"  First 5 values: {table_df['Value'].head(5).tolist()}")
    
    # Write the modified table back to the file
    RasUnsteady.write_table_to_file(new_unsteady_file, table_name, table_df, start_line)
    print(f"\nUpdated table written back to the unsteady flow file.")
    
    # Re-read the table to verify changes
    lines = open(new_unsteady_file, 'r').readlines()
    updated_table_df = RasUnsteady.parse_fixed_width_table(lines, start_line, end_line)
    print(f"\nVerified updated table statistics:")
    print(f"  Number of values: {len(updated_table_df)}")
    print(f"  Min value: {updated_table_df['Value'].min()}")
    print(f"  Max value: {updated_table_df['Value'].max()}")
    print(f"  First 5 values: {updated_table_df['Value'].head(5).tolist()}")
else:
    print("No flow hydrograph tables found in the unsteady flow file.")

## Step 8: Applying the Updated Unsteady Flow to a Plan

Now that we've modified an unsteady flow file, let's create a plan that uses it, and compute the results.

In [ ]:
# Clone an existing plan
new_plan_number = RasPlan.clone_plan("01", new_plan_shortid="Modified Flow Test")
print(f"New plan created: {new_plan_number}")

# Set the modified unsteady flow for the new plan
RasPlan.set_unsteady(new_plan_number, new_unsteady_number)
print(f"Set unsteady flow {new_unsteady_number} for plan {new_plan_number}")

# Update the plan description
new_description = "Test plan using modified unsteady flow\nFlow scaled to 75% of original\nWith restart file enabled"
RasPlan.update_plan_description(new_plan_number, new_description)
print(f"Updated plan description for plan {new_plan_number}")

# Set computation options
RasPlan.set_num_cores(new_plan_number, 2)
RasPlan.update_plan_intervals(
    new_plan_number,
    computation_interval="1MIN",
    output_interval="15MIN",
    mapping_interval="1HOUR"
)
print(f"Updated computation settings for plan {new_plan_number}")

# Compute the plan
print(f"\nComputing plan {new_plan_number} with modified unsteady flow...")
success = RasCmdr.compute_plan(new_plan_number)

if success:
    print(f"Plan {new_plan_number} computed successfully")
    
    # Check the results path
    results_path = RasPlan.get_results_path(new_plan_number)
    if results_path:
        print(f"Results available at: {results_path}")
        
        # If it exists, get its size
        results_file = Path(results_path)
        if results_file.exists():
            size_mb = results_file.stat().st_size / (1024 * 1024)
            print(f"Results file size: {size_mb:.2f} MB")
    else:
        print("No results found.")
else:
    print(f"Failed to compute plan {new_plan_number}")

## Summary of Unsteady Flow Operations

In this notebook, we've covered the following unsteady flow operations using RAS Commander:

1. **Project Initialization**: We initialized a HEC-RAS project to work with
2. **Boundary Extraction**: We extracted boundary conditions and tables from unsteady flow files
3. **Boundary Analysis**: We inspected and understood boundary condition structures
4. **Flow Title Updates**: We modified the title of an unsteady flow file
5. **Restart Settings**: We configured restart file settings for continuing simulations
6. **Table Extraction**: We extracted flow tables for analysis
7. **Table Modification**: We modified a flow table and wrote it back to the file
8. **Application**: We created a plan using our modified unsteady flow and computed results

### Key Classes and Functions Used

- `RasUnsteady.extract_boundary_and_tables()`: Extract boundary conditions and tables
- `RasUnsteady.print_boundaries_and_tables()`: Display formatted boundary information
- `RasUnsteady.update_flow_title()`: Modify the flow title
- `RasUnsteady.update_restart_settings()`: Configure restart options
- `RasUnsteady.extract_tables()`: Extract tables from unsteady flow files
- `RasUnsteady.identify_tables()`: Identify table locations in file
- `RasUnsteady.parse_fixed_width_table()`: Parse fixed-width tables
- `RasUnsteady.write_table_to_file()`: Write modified tables back to file

### Next Steps

To further explore unsteady flow operations with RAS Commander, consider:

1. **Advanced Flow Modifications**: Create scripts that systematically modify flow hydrographs
2. **Sensitivity Analysis**: Create variations of unsteady flows to assess model sensitivity
3. **Batch Processing**: Process multiple unsteady flow files for scenario analysis
4. **Custom Boundary Conditions**: Create unsteady flows from external data sources
5. **Results Analysis**: Compare results from different unsteady flow scenarios

These advanced topics can be explored by building on the foundation established in this notebook.